In [4]:
#Installing libraries

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Data Preperation.

In [6]:
Movie = pd.read_csv("Dataset/tmdb_5000_movies.csv")
Credit = pd.read_csv("Dataset/tmdb_5000_credits.csv")

In [7]:
Movie.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [8]:
Credit.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [9]:
Movie = Movie.merge(Credit, on='title')

In [10]:
Movie.columns
# Movie['production_companies']

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

To drop
budget, homepage, id, keywords, oriinal title, populariy, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, vote_average, vote_count, movie_id

In [11]:
col_drop = ['budget', 'homepage', 'keywords', 'original_language', 'original_title', 'popularity', 'production_countries', 'release_date', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count', 'movie_id', 'revenue', 'runtime']
Movie.drop(columns=col_drop, inplace=True, axis=1)

In [12]:
Movie.columns

Index(['genres', 'id', 'overview', 'production_companies', 'title', 'cast',
       'crew'],
      dtype='object')

Converting the data in a presentable format.

In [13]:
def extract1(obj):
    li = ""
    for i in eval(obj):
        # print(i)
        li += i["name"] + " "
    return li

In [14]:
Movie['genres'] = Movie['genres'].apply(extract1)

In [15]:
Movie['genres']

0       Action Adventure Fantasy Science Fiction 
1                       Adventure Fantasy Action 
2                         Action Adventure Crime 
3                    Action Crime Drama Thriller 
4               Action Adventure Science Fiction 
                          ...                    
4804                       Action Crime Thriller 
4805                              Comedy Romance 
4806               Comedy Drama Romance TV Movie 
4807                                             
4808                                 Documentary 
Name: genres, Length: 4809, dtype: object

In [16]:
Movie['production_companies'] = Movie['production_companies'].apply(extract1)

In [17]:
def extract2(obj, num):
    num = int(num)
    li = ""
    for i in eval(obj):
        if num==0:
            break
        li += (i["name"]) + " "
        num -=1
    return li

In [18]:
Movie['cast']

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4804    [{"cast_id": 1, "character": "El Mariachi", "c...
4805    [{"cast_id": 1, "character": "Buzzy", "credit_...
4806    [{"cast_id": 8, "character": "Oliver O\u2019To...
4807    [{"cast_id": 3, "character": "Sam", "credit_id...
4808    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4809, dtype: object

In [19]:
Movie['cast'] = Movie['cast'].apply(extract2, args=(3, ))

In [20]:
Movie['cast']

0         Sam Worthington Zoe Saldana Sigourney Weaver 
1            Johnny Depp Orlando Bloom Keira Knightley 
2             Daniel Craig Christoph Waltz Léa Seydoux 
3             Christian Bale Michael Caine Gary Oldman 
4           Taylor Kitsch Lynn Collins Samantha Morton 
                             ...                       
4804    Carlos Gallardo Jaime de Hoyos Peter Marquardt 
4805          Edward Burns Kerry Bishé Marsha Dietlein 
4806            Eric Mabius Kristin Booth Crystal Lowe 
4807             Daniel Henney Eliza Coupe Bill Paxton 
4808     Drew Barrymore Brian Herzlinger Corey Feldman 
Name: cast, Length: 4809, dtype: object

In [21]:
Movie['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [22]:
def extract3(obj):
    li = ""
    for i in eval(obj):
        if i['department'] == 'Directing':
            li += (i['name']) + " "
            break
    return li


In [23]:
Movie['crew'] = Movie['crew'].apply(extract3)

In [24]:
Movie['crew']

0           James Cameron 
1          Gore Verbinski 
2              Sam Mendes 
3       Christopher Nolan 
4          Andrew Stanton 
               ...        
4804     Robert Rodriguez 
4805         Edward Burns 
4806          Scott Smith 
4807          Daniel Hsia 
4808     Brian Herzlinger 
Name: crew, Length: 4809, dtype: object

In [25]:
# Genre and production companies
Movie.columns

Index(['genres', 'id', 'overview', 'production_companies', 'title', 'cast',
       'crew'],
      dtype='object')

Following this one currently-: Another approach is to make them all string by default and combine preprocess then list and vectorise.

Dropped this one-: Convert everything in a list and then concatinate talking about stop words, porter words, punctuation combine all in a list and then convert to str and then apply preprocessing.

In [26]:
Movie['tags'] = Movie["overview"] + Movie['production_companies'] + Movie["cast"] + Movie['crew'] + Movie['genres']

In [27]:
Movie['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment Sam Worthington Zoe Saldana Sigourney Weaver James Cameron Action Adventure Fantasy Science Fiction '

In [28]:
Movie['trial'] = Movie['title'].str.split()
Movie['trial']

0                                               [Avatar]
1       [Pirates, of, the, Caribbean:, At, World's, End]
2                                              [Spectre]
3                             [The, Dark, Knight, Rises]
4                                         [John, Carter]
                              ...                       
4804                                      [El, Mariachi]
4805                                         [Newlyweds]
4806                       [Signed,, Sealed,, Delivered]
4807                                 [Shanghai, Calling]
4808                              [My, Date, with, Drew]
Name: trial, Length: 4809, dtype: object

I have to apply now datapreprocessing an then vectors and continue.

Total


def clean_list_of_text(
        input_text, 
        enable_stopword_removal=True,
        enable_punctuation_removal=True,
        enable_lemmatization=True,
        enable_stemming=False
    ):
    # Get list of operations
    enabled_operations = [word_tokenize]
    if enable_stopword_removal:
        enabled_operations.append(remove_stopwords)
    if enable_punctuation_removal:
        enabled_operations.append(remove_punctuation)
    if enable_lemmatization:
        enabled_operations.append(lemmatize)
    if enable_stemming:
        enabled_operations.append(stem)
    print(f'Enabled Operations: {len(enabled_operations)}')
    

    # Run all operations
    cleaned_text_lines = input_text
    for operation in enabled_operations:
        # Run for all lines
        cleaned_text_lines = [operation(line) for line in cleaned_text_lines]
    
    return cleaned_text_lines

# Example of applying the function
clean_list_of_text(sample_lines, enable_stopword_removal=True, enable_punctuation_removal=True, enable_lemmatization=True)


Input string or list?


# For removing stop words

import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
print(STOPWORDS)

new_text = remove_stopwords(text)



# For punctuation-: 

import string

def remove_punctuation(input_text):
    return [token for token in input_text if token not in set(string.punctuation)]

Apply punctuation function
tokens_without_punctuation = [remove_punctuation(line) for line in tokens_without_stopwords]


# For stem words-:

from nltk.stem import SnowballStemmer

Instantiate and test on one word
stemmer = SnowballStemmer('english')
stemmer.stem('sweeping')

def stem(input_text):
    stemmer = SnowballStemmer('english')
    return [stemmer.stem(token) for token in input_text]

Apply stemming function
tokens_stemmed = [stem(line) for line in tokens_without_punctuation]

## Data preprocessing pipeline.

In [29]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
print(stemmer.stem("dancing"))

danc


In [30]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    if isinstance(text, str):  # Check if 'text' is a string
        text = "".join(word.lower() for word in text if word not in string.punctuation)
        text = " ".join(word for word in text.split() if word not in stop_words)  # Use split to tokenize
        text = " ".join(stemmer.stem(word) for word in text.split())  # Use split to tokenize
        return text
    elif isinstance(text, float):  # Check if 'text' is a float
        return ""  # Return an empty string for float values
    else:
        return str(text)  # Convert non-string values to string


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
(Movie['tags'][500])

'It\'s a major double-cross when former police officer Brian O\'Conner teams up with his ex-con buddy Roman Pearce to transport a shipment of "dirty" money for shady Miami-based import-export dealer Carter Verone. But the guys are actually working with undercover agent Monica Fuentes to bring Verone down.Mikona Productions GmbH & Co. KG Universal Pictures Original Film Ardustry Entertainment Paul Walker Tyrese Gibson Eva Mendes John Singleton Action Crime Thriller '

In [32]:
Movie["tags"] = Movie["tags"].apply(preprocess_text)

In [ ]:
Movie["tags"][500]

'major doublecross former polic offic brian oconn team excon buddi roman pearc transport shipment dirti money shadi miamibas importexport dealer carter veron guy actual work undercov agent monica fuent bring veron downmikona product gmbh co kg univers pictur origin film ardustri entertain paul walker tyres gibson eva mend john singleton action crime thriller'

In [ ]:
# Movie["tags"] = Movie["tags"].str.split()

In [ ]:
vector = CountVectorizer(max_features=5000, stop_words='english')
text_data = Movie['tags'].values.tolist()
bow_vector = vector.fit_transform(text_data)

In [ ]:
Movie['bow_vector'] = bow_vector.toarray().tolist()

Compute cosine similarity and start making the predictions.

In [ ]:
Movie.columns
Movie.drop(['genres', "overview", "production_companies", "cast", "crew", "trial"], inplace=True, axis=1)

In [ ]:
Movie.columns

Index(['id', 'title', 'tags', 'bow_vector'], dtype='object')

In [ ]:
Movie['bow_vector'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
def compute_similarity(vector1, vector2):
    if np.any(vector1) and np.any(vector2):
        vector1 = np.array(vector1).reshape(1, -1)  # Reshape to 2D array with one row
        vector2 = np.array(vector2).reshape(1, -1)
        return cosine_similarity(vector1, vector2)[0][0]
    else:
        return 0.0

In [ ]:
similarity_info = []
i = 5
for idx, movie_row in Movie.iterrows():
    movie_id = movie_row['id']
    print("For ", movie_id)
    movie_name = movie_row['title']   
    vector1 = movie_row['bow_vector']
    # print(vector1.shape)
    # break
    # vector1.reshape(1, -1)
    similarities = []
    for _, other_row in Movie.iterrows():
        other_movie_id = other_row['id']
        other_movie_name = other_row['title']
        print("and ", other_movie_name)
        vector2 = other_row['bow_vector']
        # vector2.reshape(-1, 1)
        similarity = compute_similarity(vector1, vector2)
        print(movie_name, " ", other_movie_id, " similarity is ", similarity)
        similarities.append((other_movie_id, other_movie_name, similarity))
    similarities.sort(key=lambda x: x[2], reverse=True)
    similarity_info.append((movie_id, similarities))
Movie['similarity_column'] = similarity_info

For  19995
and  Avatar
Avatar   19995  similarity is  1.0
and  Pirates of the Caribbean: At World's End
Avatar   285  similarity is  0.13803940759355818
and  Spectre
Avatar   206647  similarity is  0.0828236445561349
and  The Dark Knight Rises
Avatar   49026  similarity is  0.11447797027778953
and  John Carter
Avatar   49529  similarity is  0.12346619958119871
and  Spider-Man 3
Avatar   559  similarity is  0.1583891032045364
and  Tangled
Avatar   38757  similarity is  0.0
and  Avengers: Age of Ultron
Avatar   99861  similarity is  0.13386322447594806
and  Harry Potter and the Half-Blood Prince
Avatar   767  similarity is  0.11219854269064289
and  Batman v Superman: Dawn of Justice
Avatar   209112  similarity is  0.24398510380867344
and  Superman Returns
Avatar   1452  similarity is  0.11640504929492972
and  Quantum of Solace
Avatar   10764  similarity is  0.11772040233700279
and  Pirates of the Caribbean: Dead Man's Chest
Avatar   58  similarity is  0.1450036976414354
and  The Lone Ran

In [ ]:
similarity_info[1][1][1]

In [ ]:
for i in range(5):
    for j in range(5):
        print("for",similarity_info[i][1][0], similarity_info[i][1][j])

In [ ]:
Movie.to_csv('NewMovies', index=False)